# Covariance Matrix Estimation Under Heteroskedasticity

Reference: Bruce Hansen, "Econometrics", 2021. 

### Regression with formula

In [1]:
dat <- read.table("./data/cps09mar.txt")
experience <- dat[,1]-dat[,4]-6
mbf <- (dat[,11]==2)&(dat[,12]<=2)&(dat[,2]==1)&(experience==12) 
sam <- (dat[,11]==4)&(dat[,12]==7)&(dat[,2]==0)
dat1 <- dat[mbf,]
dat2 <- dat[sam,]
y <- as.matrix(log(dat2[,5]/(dat2[,6]*dat2[,7])))
experience <- dat2[,1]-dat2[,4]-6
exp2 <- (experience^2)/100
const <-matrix(1,nrow(dat2),1) #Intercept
educ <-dat2[,4] #Education
x <- cbind(const,educ,experience,exp2) 
xx <- t(x)%*%x
xy <- t(x)%*%y
beta <- solve(xx,xy)
print(beta)

                  [,1]
            0.57535630
educ        0.14331647
experience  0.03557892
exp2       -0.07137808


### Let's check with R's `lm()` function

In [2]:
olsreg1 <- lm(y~x[,2:4]) #Note that I am removing the constant term, first column
summary(olsreg1)


Call:
lm(formula = y ~ x[, 2:4])

Residuals:
     Min       1Q   Median       3Q      Max 
-1.80706 -0.29973  0.03816  0.35345  2.73619 

Coefficients:
                   Estimate Std. Error t value Pr(>|t|)    
(Intercept)         0.57536    0.18683   3.080  0.00229 ** 
x[, 2:4]educ        0.14332    0.01163  12.322  < 2e-16 ***
x[, 2:4]experience  0.03558    0.01086   3.277  0.00119 ** 
x[, 2:4]exp2       -0.07138    0.02957  -2.414  0.01647 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.5608 on 264 degrees of freedom
Multiple R-squared:  0.3876,	Adjusted R-squared:  0.3806 
F-statistic: 55.69 on 3 and 264 DF,  p-value: < 2.2e-16


### Parameters

In [3]:
n <- nrow(y)
k <- ncol(x)

### OLS errors, variance and other useful matrices

In [4]:
e <- y-x%*%beta
xx <- t(x)%*%x
xxinv <- solve(t(x)%*%x)
sig2 <- as.vector((t(e) %*% e)/(n-k))
print(sig2)
print(xxinv)


[1] 0.3144554
                                 educ    experience          exp2
            0.111002721 -6.345889e-03 -1.825163e-03  0.0032141380
educ       -0.006345889  4.301833e-04 -2.013356e-05  0.0001076635
experience -0.001825163 -2.013356e-05  3.748921e-04 -0.0009682789
exp2        0.003214138  1.076635e-04 -9.682789e-04  0.0027809544


The general form for the covariance matrix is
$$
Var(\hat{\beta}) = (\mathbf{X'X})^{-1} (\mathbf{X' \Omega X}) (\mathbf{X'X})^{-1}
$$
where $ \mathbf{\Omega} $ is unknown. 

### Homoskedastic formula  

$$
Var(\mathbf{\hat{\beta}}) = \sigma^2 (\mathbf{X'X})^{-1}
$$
since
$$
E[\mathbf{e \: e' | X}] = \sigma^2 \mathbf{I}
$$

In [5]:
v <- xxinv*sig2
s <- sqrt(diag(v))
t <- beta/s
olsh <- cbind(beta,s,t)
print(olsh)

                                s          
            0.57535630 0.18682987  3.079573
educ        0.14331647 0.01163071 12.322244
experience  0.03557892 0.01085757  3.276876
exp2       -0.07137808 0.02957171 -2.413728


## Heteroskedasticity-consistent Covariance matrix estimators

Ideally we would estimate the general form of the covariance matrix like this:
$$
Var(\hat{\beta})^{\textrm{ideal}} = (\mathbf{X'X})^{-1} \left( \sum_{i=1}^{n} X_i X_i' e_i^2  \right) (\mathbf{X'X})^{-1}
$$
Since the errors $e^2$ are unobserved, the ideal V above is not a feasible estimator. However, we can replace the errors with the least squares residuals $\hat{e}_i$. 

### HC0

Substituting $\hat{e}_i$ in the "ideal" formula above, we get
$$
Var(\hat{\beta})^{\textrm{HC0}} = (\mathbf{X'X})^{-1} \left( \sum_{i=1}^{n} X_i X_i' \hat{e}_i^2  \right) (\mathbf{X'X})^{-1}
$$
The label *HC* refers to "heteroskedasticity-consistent." The label *HC0* refers to this being the baseline
heteroskedasticity-consistent covariance matrix estimator.

In [6]:
u0 <- x*(e%*%matrix(1,1,k)) # x*e
v0 <- xxinv %*% (t(u0)%*%u0) %*% xxinv
s0 <- sqrt(diag(v0)) 
t0 <- beta/s0
ols0 <- cbind(beta,s0,t0)
print(ols0)

                               s0          
            0.57535630 0.19362680  2.971470
educ        0.14331647 0.01152244 12.438031
experience  0.03557892 0.01121874  3.171382
exp2       -0.07137808 0.02918124 -2.446026


The necessary libraries to use Robust Standard errors in `R` are `sandwich` and `lmtest`. Let's try and run these to see if we get the same as our "by-hand" approach.

In [9]:
library(zoo)
library(sandwich)
library(lmtest)

In [10]:
fm0 <- lm(y ~ x)
coeftest(fm0, vcov = vcovHC(fm0, type = "HC0"))


t test of coefficients:

             Estimate Std. Error t value  Pr(>|t|)    
(Intercept)  0.575356   0.193627  2.9715  0.003237 ** 
xeduc        0.143316   0.011522 12.4380 < 2.2e-16 ***
xexperience  0.035579   0.011219  3.1714  0.001696 ** 
xexp2       -0.071378   0.029181 -2.4460  0.015097 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


### HC1

Since $\hat{e}_i$ is biased towards zero, we can estimate the variance $\sigma^2$, we can use $s^2$ to 
scale the moment estimator $\hat{\sigma}^2$ by $n/(n−k)$. 
$$
Var(\hat{\beta})^{\textrm{HC1}} = \left( \frac{n}{n-k} \right) (\mathbf{X'X})^{-1} \left( \sum_{i=1}^{n} X_i X_i' \hat{e}_i^2  \right) (\mathbf{X'X})^{-1}
$$

In [11]:
a <- n/(n-k)
u1 <- u0
v1 <- a * xxinv %*% (t(u1)%*%u1) %*% xxinv 
s1 <- sqrt(diag(v1))
t1 <- beta/s1
ols1 <- cbind(beta,s1,t1)
print(ols1)

                               s1          
            0.57535630 0.19508816  2.949212
educ        0.14331647 0.01160940 12.344861
experience  0.03557892 0.01130341  3.147626
exp2       -0.07137808 0.02940148 -2.427704


In [12]:
fm1 <- lm(y ~ x)
coeftest(fm1, vcov = vcovHC(fm1, type = "HC1"))


t test of coefficients:

             Estimate Std. Error t value  Pr(>|t|)    
(Intercept)  0.575356   0.195088  2.9492  0.003471 ** 
xeduc        0.143316   0.011609 12.3449 < 2.2e-16 ***
xexperience  0.035579   0.011303  3.1476  0.001835 ** 
xexp2       -0.071378   0.029401 -2.4277  0.015864 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


Though the scaling by $n/(n-k)$ is *ad hoc*, *HC1* is often recommended over the unscaled *HC0*

### HC2

Alternatively, we can use standardised residuals, say $\tilde{e}_i$ or the prediction error, which implies
$$
Var(\hat{\beta})^{\textrm{HC2}} = (\mathbf{X'X})^{-1} \left( \sum_{i=1}^{n} X_i X_i' \tilde{e}_i^2  \right) (\mathbf{X'X})^{-1}
$$
or
$$
Var(\hat{\beta})^{\textrm{HC2}} = (\mathbf{X'X})^{-1} \left( \sum_{i=1}^{n} (1-h_{ii})^{-1} X_i X_i' \hat{e}_i^2  \right) (\mathbf{X'X})^{-1}
$$
where $h_{ii}$ is element $(i,i)$ of the leverage measure. The leverage values for $\mathbf{X}$ are the diagonal elements of the projection matrix $\mathbf{P_X = X (X'X)^{-1} X'}$:
$$
h_{ii} = X_i' (\mathbf{X'X})^{-1} X_i
$$
The leverage value $h_{ii}$ is a normalized length of the observed regressor vector $X_i$. They appear fre- quently in leave-one-out re-gression, influential observations, robust covariance matrix estimation, and cross-validation.

In [13]:
leverage <- rowSums(x*(x%*%xxinv))
u2 <- x*((e/sqrt(1-leverage))%*%matrix(1,1,k)) 
v2 <- xxinv %*% (t(u2)%*%u2) %*% xxinv
s2 <- sqrt(diag(v2)) 
t2 <- beta/s2
ols2 <- cbind(beta,s2,t2)
print(ols2)

                               s2          
            0.57535630 0.19702185  2.920266
educ        0.14331647 0.01169374 12.255831
experience  0.03557892 0.01178237  3.019675
exp2       -0.07137808 0.03150154 -2.265860


In [14]:
fm2 <- lm(y ~ x)
coeftest(fm2, vcov = vcovHC(fm1, type = "HC2"))


t test of coefficients:

             Estimate Std. Error t value  Pr(>|t|)    
(Intercept)  0.575356   0.197022  2.9203  0.003799 ** 
xeduc        0.143316   0.011694 12.2558 < 2.2e-16 ***
xexperience  0.035579   0.011782  3.0197  0.002778 ** 
xexp2       -0.071378   0.031502 -2.2659  0.024270 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


### HC3

$$
Var(\hat{\beta})^{\textrm{HC3}} = (\mathbf{X'X})^{-1} \left( \sum_{i=1}^{n} (1-h_{ii})^{-2} X_i X_i' \hat{e}_i^2  \right) (\mathbf{X'X})^{-1}
$$

In [15]:
u3 <- x*((e/(1-leverage))%*%matrix(1,1,k))
v3 <- xxinv %*% (t(u3)%*%u3) %*% xxinv
s3 <- sqrt(diag(v3)) 
t3 <- beta/s3
ols3 <- cbind(beta,s3,t3)
print(ols3)

                               s3          
            0.57535630 0.20102036  2.862179
educ        0.14331647 0.01187627 12.067461
experience  0.03557892 0.01254629  2.835812
exp2       -0.07137808 0.03459159 -2.063452


In [16]:
fm3 <- lm(y ~ x)
coeftest(fm3, vcov = vcovHC(fm3, type = "HC1"))


t test of coefficients:

             Estimate Std. Error t value  Pr(>|t|)    
(Intercept)  0.575356   0.195088  2.9492  0.003471 ** 
xeduc        0.143316   0.011609 12.3449 < 2.2e-16 ***
xexperience  0.035579   0.011303  3.1476  0.001835 ** 
xexp2       -0.071378   0.029401 -2.4277  0.015864 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


### What to use in practice?

*HC0, HC1, HC2 and HC3* are called *robust, heteroskedasticity-
consistent, or heteroskedasticity-robust covariance matrix* estimators. The *HC0* estimator was developed by Eicker (1963) and then introduced by White (1980) and is sometimes called the Eicker-White or White covariance matrix estimator. Note that
$$
Var(\hat{\beta})^{\textrm{HC0}} <Var(\hat{\beta})^{\textrm{HC2}} <Var(\hat{\beta})^{\textrm{HC3}} 
$$
*HC2* and *HC3* are recommended in practice, however. *HC2* is unbiased under homoskedasticity and *HC3* is conservative for any *X* . Note that in most cases *HC1, HC2 and HC3* will be very similar, unless  the sample has a large leverage value *h* for some observation (or multiple large leverage values).

## Clustered Standard Errors

We'll use `tidyverse` to read the *.csv* file.

In [17]:
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.3     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [20]:
# Load the data and create variables
data <- read_csv("./data/DDK2011.csv") 
y <- scale(as.matrix(data$totalscore))
n <- nrow(y)
x <- cbind(matrix(1,n,1),as.matrix(data$tracking)) 
schoolidmat <- as.matrix(data$schoolid)
k <- ncol(x)
xx <- t(x)%*%x
invx <- solve(xx)
beta <- solve(xx,t(x)%*%y)
print(beta)
fm_clustered<-lm(y~data$tracking)
summary(fm_clustered)

Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Rows: 5795 Columns: 62
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (40): district, division, zone, girl, agetest, stream_meanpercentile, SD...
dbl (22): pupilid, schoolid, bungoma, tracking, sbm, etpteacher, lowstream, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


            [,1]
[1,] -0.07093386
[2,]  0.13789390



Call:
lm(formula = y ~ data$tracking)

Residuals:
    Min      1Q  Median      3Q     Max 
-1.4842 -0.7973 -0.1667  0.6299  3.3746 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)   -0.07093    0.01881  -3.771 0.000164 ***
data$tracking  0.13789    0.02622   5.258  1.5e-07 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.9977 on 5793 degrees of freedom
Multiple R-squared:  0.004751,	Adjusted R-squared:  0.004579 
F-statistic: 27.65 on 1 and 5793 DF,  p-value: 1.505e-07


### Clustered robust standard error

Let's start again with the general form for the covariance matrix
$$
Var(\hat{\beta}) = (\mathbf{X'X})^{-1} (\mathbf{X' \Omega_n X}) (\mathbf{X'X})^{-1}
$$
where $ \mathbf{\Omega_n} $ is the clustered variance-covariance and is unknown. Let's assume that there are $G$ clusters, $g=1,\ldots,G$. Now let's consider the covariance matrix of $\hat{\beta}$, let $\Sigma_g = E \left[\mathbf{e_g e_g' | X_g}\right]$ be the $n_g \times n_g$ conditional matrix of the errors within the $g^{th}$ cluster. Assume that the observations across clusters are independent of each other then
$$
Var \left[ \left( \sum_{g=1}^{G} \mathbf{X_g' e_g } \right)  | \mathbf{X_g} \right]  = \sum_{g=1}^{G} Var \left[ \mathbf{X_g' e_g | X_g}  \right]
$$
When solving this expression, we obtain 
$$
\Omega_n \overset{def}{=} \sum_{g=1}^G \mathbf{X_g' \Sigma_g X_g}
$$
This formula allows for correlation within clustered. A simplified version of this formula can be found by assuming independence of observations within the clusters. Arellano (1987) proposed a cluster-robust covariance matrix estimator which is an extension of the White estimator. Just as with the White covariance estimator, we can use the squared error $e_i^2$ as an unbiased estimate. But the problem is that the errors are not observed. Hence we can use the squared OLS residuals instead:
$$
\hat{\Omega}_n = \sum_{g=1}^{G} \mathbf{X_g' \hat{e}_g \hat{e}_g' X_g} = \sum_{g=1}^{G} \left( \sum_{i=1}^{n_g} X_{ig} \hat{e}_{ig} \right) \left( \sum_{l=1}^{n_g} X_{lg} \hat{e}_{lg} \right)'
$$
and now
$$
Var(\hat{\beta}) = \mathbf{a}_n (\mathbf{X'X})^{-1} (\mathbf{X' \Omega_n X}) (\mathbf{X'X})^{-1}
$$
with the finite sample adjustment term ${a}_n$:
$$
{a}_n = \frac{(n-1)}{(n-k)}\frac{G}{(G-1)}
$$
The factor $G /(G − 1)$ is from Chris Hansen (2007) in the context of equal-sized clusters to improve performance when the number of clusters $G$ is small. The factor $(n−1)/(n−k)$ is an ad hoc generalization which nests the adjustment used *HC1* since $G=n$ implies the simplification $a_n =n/(n−k)$.

We need to choose a cluster. Let's cluster on `schoolid`:

In [21]:
xe <- x*rep(y-x%*%beta,times=k) # X*ehat
xe_sum <- rowsum(xe,schoolidmat) # sum_i X*ehat by group = schoolid
G <- nrow(xe_sum)
omega <- t(xe_sum)%*%xe_sum # Omegahat = sum_g (sum_i X*ehat)*(sum_i X*ehat)'
scale <- G/(G-1)*(n-1)/(n-k) #scale a =
V_clustered <- scale*invx%*%omega%*%invx 
se_clustered <- sqrt(diag(V_clustered))
t_clustered <- beta / se_clustered 
ols_clustered <- cbind(beta,se_clustered,t_clustered)
print(se_clustered)
print(ols_clustered)

[1] 0.05434114 0.07718409
                 se_clustered          
[1,] -0.07093386   0.05434114 -1.305344
[2,]  0.13789390   0.07718409  1.786559


We can use `sandwich` and `lmtest` to cluster the standard errors in `R`. Note that you need to specify the *id* or *variable* to cluster on.

In [22]:
fm_c <- lm(y ~ data$tracking, data=data)
coeftest(fm_c, vcov = vcovCL, cluster = ~data$schoolid)


t test of coefficients:

               Estimate Std. Error t value Pr(>|t|)  
(Intercept)   -0.070934   0.054341 -1.3053  0.19183  
data$tracking  0.137894   0.077184  1.7866  0.07406 .
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1
